#### Projeto Python IA: Inteligência Artificial e Previsões

## Case: Score de Crédito dos Clientes

Autor: Raul Lemelle

*11 Janeiro 2024*

**Visão geral:** Conseguir definir para o banco o score de crédito dos clientes. Analisar todos os clientes do banco e, com base nessa análise, criar um modelo que consiga ler as informações dos clientes e dizer automaticamente o score de crédito dele: Good, Standard, Poor.

**Objetivo:** Realizar previsão do score de crédito de novos clientes.

Antes de começar, vamos importar alguns pacotes e bibliotecas.

In [ ]:
!pip install scikit-learn

In [1]:
import csv # manipulação
import requests # manipulação
import pandas as pd # manipulação

from io import StringIO # manipulação
from sklearn.preprocessing import LabelEncoder # IA
from sklearn.model_selection import train_test_split # IA
from sklearn.ensemble import RandomForestClassifier # IA
from sklearn.neighbors import KNeighborsClassifier # IA
from sklearn.metrics import accuracy_score # IA


A nossa base de dados, denominada "clientes.csv", encontra-se hospedada em um repositório no [meu perfil do GitHub.](https://github.com/Raul-Lemelle)

In [2]:
url_clientes = 'https://raw.githubusercontent.com/Raul-Lemelle/data/main/Aula%203/clientes.csv'

In [3]:
def obter_DataFrame_csv(url):
    # Solicitação para obter o arquivo
    response = requests.get(url)

    # Verificando solicitação -> bem-sucedida (código de status 200)
    if response.status_code == 200:
        # StringIO para criar um objeto de arquivo temporário
        csv_data = StringIO(response.text)

        # Criar um DataFrame
        cliente_df = pd.read_csv(csv_data)
        return cliente_df
    else:
        # Solicitação -> não foi bem-sucedida
        print(f"Falha na solicitação. Código de status: {response.status_code}")
        return None

In [4]:
def codificar_colunas_objeto(dataframe):
    codificador = LabelEncoder()

    for coluna in dataframe.columns:
        if dataframe[coluna].dtype == "object" and coluna != "score_credito":
            dataframe[coluna] = codificador.fit_transform(dataframe[coluna])
    return dataframe

In [5]:
clientes_df = obter_DataFrame_csv(url_clientes)

**Vamos realizar algumas análises no DataFrame com o objetivo de aprimorar nossas percepções.**

In [6]:
# Análise inicial do Dataframe
clientes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  object 
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  object 
 13  divida_total              100000 non-null  fl

In [7]:
# Visualizando nosso DataFrame
clientes_df.head()

,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,3392,1,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,265.0,21.46538,alto_gasto_pagamento_baixos,312.494089,Good,1,1,1,1,0
1,3392,2,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,266.0,21.46538,baixo_gasto_pagamento_alto,284.629162,Good,1,1,1,1,0
2,3392,3,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,267.0,21.46538,baixo_gasto_pagamento_medio,331.209863,Good,1,1,1,1,0
3,3392,4,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,5.0,...,268.0,21.46538,baixo_gasto_pagamento_baixo,223.451310,Good,1,1,1,1,0
4,3392,5,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,6.0,...,269.0,21.46538,alto_gasto_pagamento_medio,341.489231,Good,1,1,1,1,0


In [8]:
# Verificação de dados faltantes.
clientes_df.isna().any()

id_cliente                  False
mes                         False
idade                       False
profissao                   False
salario_anual               False
num_contas                  False
num_cartoes                 False
juros_emprestimo            False
num_emprestimos             False
dias_atraso                 False
num_pagamentos_atrasados    False
num_verificacoes_credito    False
mix_credito                 False
divida_total                False
taxa_uso_credito            False
idade_historico_credito     False
investimento_mensal         False
comportamento_pagamento     False
saldo_final_mes             False
score_credito               False
emprestimo_carro            False
emprestimo_casa             False
emprestimo_pessoal          False
emprestimo_credito          False
emprestimo_estudantil       False
dtype: bool


Observamos a ausência de dados faltantes em nosso DataFrame, estabelecendo, portanto, uma base sólida para uma análise precisa em nosso estudo.

Para empregar a biblioteca de IA que facilitará nossas previsões, é necessário converter as colunas categóricas para numéricas. Utilizaremos o próprio codificador incorporado na biblioteca de IA para executar essa operação. Chamamos a função codificar_colunas_objeto definida acima.

In [9]:
clientes_df = codificar_colunas_objeto(clientes_df)

In [10]:
# Visualização do novo DataFrame.
clientes_df.head()

,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,3392,1,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,265.0,21.46538,1,312.494089,Good,1,1,1,1,0
1,3392,2,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,266.0,21.46538,3,284.629162,Good,1,1,1,1,0
2,3392,3,23.0,2,19114.12,3.0,4.0,3.0,4.0,3.0,...,267.0,21.46538,5,331.209863,Good,1,1,1,1,0
3,3392,4,23.0,2,19114.12,3.0,4.0,3.0,4.0,5.0,...,268.0,21.46538,4,223.451310,Good,1,1,1,1,0
4,3392,5,23.0,2,19114.12,3.0,4.0,3.0,4.0,6.0,...,269.0,21.46538,2,341.489231,Good,1,1,1,1,0


**Aplicação do modelo IA**

Vamos determinar quais colunas serão utilizadas no treinamento do modelo.

A variável "y" será a coluna que o modelo tentará prever, sendo o "score de crédito" ("score_credito").

A variável "x" consistirá nas colunas que empregaremos para antecipar o score de crédito. Remover a coluna "id_cliente" será benéfico para nosso conjunto de dados, já que se trata de um número aleatório, resultando em um DataFrame mais leve e otimizando as operações.

In [11]:
x = clientes_df.drop(["score_credito", "id_cliente"], axis=1)
y = clientes_df["score_credito"]

Dividimos os dados em conjuntos de treinamento e teste. O conjunto de treinamento será fornecido aos modelos para aprendizado, enquanto o conjunto de teste será empregado para avaliar se o modelo aprendeu de maneira precisa.

In [12]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state=1)

Com base na biblioteca [scikit-learn](https://scikit-learn.org/stable/index.html) - Machine Learning in Python:

Optaremos pelo modelo de Árvore de Decisão ([RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)), em que cada decisão é fundamentada na comparação de dois números em tempo constante.

Além disso, empregaremos o modelo KNN  ([K-nearest neighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)), que busca classificar cada amostra de um conjunto de dados avaliando sua distância em relação aos vizinhos mais próximos.

In [13]:
modelo_arvore = RandomForestClassifier()
modelo_knn = KNeighborsClassifier()

Treinando os modelos.

In [14]:
modelo_arvore.fit(x_treino, y_treino)
modelo_knn.fit(x_treino, y_treino)

KNeighborsClassifier()

**Verificando acurácia dos modelos.**

Exemplo 1:

Se o modelo atribuísse a classe "Standard" a todas as previsões, a acurácia seria calculada como a proporção de previsões corretas em relação ao total de previsões.


In [15]:
contagem_scores = clientes_df["score_credito"].value_counts()
print(contagem_scores['Standard'] / sum(contagem_scores))

0.53174


Teriamos a acurácia de 53% nesse caso.

Calcular as previsões.

In [16]:
previsao_arvore = modelo_arvore.predict(x_teste)
previsao_knn = modelo_knn.predict(x_teste.to_numpy())

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


Vamos comparar as previsões com o y_teste.
No caso, buscamos um valor mais elevado, pois quanto maior a acurácia, melhor.

Além disso, o resultado deve superar o desempenho do nosso Exemplo 1 de 53%.

In [17]:
print(accuracy_score(y_teste, previsao_arvore))
print(accuracy_score(y_teste, previsao_knn))

0.8263333333333334
0.7324


Com isso observamos que o modelo de Árvore de Decisão tem mais acurácia.

Novas previsões.

Vamos importar do mesmo repositório Github citado acima novos_clientes.csv.

In [18]:
url_novos_clientes = "https://raw.githubusercontent.com/Raul-Lemelle/data/main/Aula%203/novos_clientes.csv"

In [19]:
novos_clientes_df = obter_DataFrame_csv(url_novos_clientes)

In [21]:
# Visualizando DataFrame
novos_clientes_df.head()

,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,num_pagamentos_atrasados,...,taxa_uso_credito,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,1,31.0,empresario,19300.340,6.0,7.0,17.0,5.0,52.0,19.0,...,29.934186,218.0,44.50951,baixo_gasto_pagamento_baixo,312.487689,1,1,0,0,0
1,4,32.0,advogado,12600.445,5.0,5.0,10.0,3.0,25.0,18.0,...,28.819407,12.0,0.00000,baixo_gasto_pagamento_medio,300.994163,0,0,0,0,1
2,2,48.0,empresario,20787.690,8.0,6.0,14.0,7.0,24.0,14.0,...,34.235853,215.0,0.00000,baixo_gasto_pagamento_alto,345.081577,0,1,0,1,0


**Novas previsões**

In [22]:
# Codificar colunas categóricas
novos_clientes_df = codificar_colunas_objeto(novos_clientes_df)

In [23]:
# Aplicando o modelo de Árvore de Decisão
previsoes = modelo_arvore.predict(novos_clientes_df)
print(previsoes)

['Poor' 'Good' 'Standard']


In [25]:
# Adicionando as previsões ao DataFrame
novos_clientes_df["score_credito"] = previsoes

In [26]:
# Visualizando o DataFrame
novos_clientes_df

,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,num_pagamentos_atrasados,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil,score_credito
0,1,31.0,1,19300.340,6.0,7.0,17.0,5.0,52.0,19.0,...,218.0,44.50951,1,312.487689,1,1,0,0,0,Poor
1,4,32.0,0,12600.445,5.0,5.0,10.0,3.0,25.0,18.0,...,12.0,0.00000,2,300.994163,0,0,0,0,1,Good
2,2,48.0,1,20787.690,8.0,6.0,14.0,7.0,24.0,14.0,...,215.0,0.00000,0,345.081577,0,1,0,1,0,Standard


Agora, iremos analisar quais colunas (características) exercem maior influência na definição do score de crédito.

In [27]:
colunas = list(x_teste.columns)
importancia = pd.DataFrame(index=colunas, data=modelo_arvore.feature_importances_)
importancia = importancia * 100
print(importancia)

                                  0
mes                        3.918536
idade                      4.234281
profissao                  3.280890
salario_anual              5.125413
num_contas                 3.345855
num_cartoes                4.441774
juros_emprestimo           7.274903
num_emprestimos            3.298904
dias_atraso                6.585797
num_pagamentos_atrasados   4.568422
num_verificacoes_credito   4.520987
mix_credito                8.680162
divida_total              12.003780
taxa_uso_credito           5.048199
idade_historico_credito    7.430858
investimento_mensal        4.908588
comportamento_pagamento    2.342226
saldo_final_mes            5.443846
emprestimo_carro           0.703263
emprestimo_casa            0.743522
emprestimo_pessoal         0.709075
emprestimo_credito         0.683533
emprestimo_estudantil      0.707185
